# Tomato Leaf Disease Detection 0.998 [Training]

### Hi kagglers, This is `training` notebook using `Keras`.

> 
>  [Tomato Leaf Disease Detection 0.998 [inference]](https://www.kaggle.com/ammarnassanalhajali/tomato-leaf-disease-detection-0-998-inference)



### Please if this kernel is useful, <font color='red'>please upvote !!</font>

# importing Libraries

In [ ]:
import os, cv2, json
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import EfficientNetB6
from tensorflow.keras.applications import InceptionV3

from tensorflow.keras.applications import ResNet50

from tensorflow.keras.optimizers import Adam

from PIL import Image

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, BatchNormalization, GlobalAveragePooling2D

# Loading Data

In [ ]:
train= pd.read_csv("../input/tomato-diseases-dataset-csvimages/train.csv")

# Splitting DataSet

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_validate, y_train, y_test = train_test_split(train, train.label, 
                                                    train_size=0.8, 
                                                    random_state=42,
                                                    stratify=train.label)

In [ ]:
df_train = df_train.reset_index(drop=True)
df_validate = df_validate.reset_index(drop=True)

# Visualizing Data

In [ ]:
sample = df_train[df_train.label == 3].sample(3)
plt.figure(figsize=(15, 5))
for ind, (img, label) in enumerate(zip(sample.img, sample.label)):
    plt.subplot(1, 3, ind + 1)
    img = cv2.imread(os.path.join("../input/tomato-diseases-dataset-csvimages/Tomato_images/Tomato_images", img))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.axis("off")
    
plt.show()

# Parameters

In [ ]:
# Main parameters
BATCH_SIZE = 16
STEPS_PER_EPOCH = len(train)*0.8 / BATCH_SIZE
VALIDATION_STEPS = len(train)*0.2 / BATCH_SIZE
EPOCHS =60 #
IMG_WIDTH= 256
IMG_HEIGHT= 256
train_dir = "../input/tomato-diseases-dataset-csvimages/Tomato_images/Tomato_images"

In [ ]:
df_train.label = df_train.label.astype('str')
df_validate.label = df_validate.label.astype('str')

# Data Generator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                               shear_range = 0.2,
                               zoom_range = 0.2,
                               rotation_range = 180,
                               vertical_flip = True,
                               horizontal_flip = True)
# our train_datagen generator will use the following transformations on the images
validation_datagen = ImageDataGenerator(rescale=1./255)



train_generator = train_datagen.flow_from_dataframe(df_train, 
                                                    train_dir,
                                                    target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    x_col='img',
                                                    y_col='label',
                                                    class_mode = 'categorical')

# generator = ImageDataGenerator(*args).flow_from_dataframe(dataframe, directory, target_size,
# batch_size, x_col, y_col, class_mode)
# your dataframe shoudl be in the format such that x_col = features, y_col = class/label
# binary class mode since output is either 0(dog) or 1(cat)

validation_generator = validation_datagen.flow_from_dataframe(df_validate, 
                                                   train_dir,
                                                    target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                    x_col='img',
                                                    y_col='label',
                                                    class_mode='categorical', 
                                                  batch_size=BATCH_SIZE)

# Creating Model

In [ ]:
def create_model():
    efficientnet_layers = InceptionV3(weights='imagenet', 
                                         include_top=False, 
                                         input_shape = (IMG_WIDTH, IMG_HEIGHT, 3),
                                         pooling='avg')

    model = Sequential()
    model.add(efficientnet_layers)
    model.add(Dense(10, activation="softmax"))
    model.compile(optimizer = Adam(lr = 0.001),
                  loss = "categorical_crossentropy",
                  metrics = ["acc"])

    return model


In [ ]:
model = create_model()
model.summary()

In [ ]:
from keras.utils import plot_model
model.summary()
# plot model architecture
plot_model(model, show_shapes=True, to_file='naive_inception_module.png')

# Training

In [ ]:
model_save = ModelCheckpoint('./InceptionV3_256.h5', 
                             save_best_only = True, 
                             save_weights_only = False,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)
early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 10, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3, 
                              patience = 2, min_delta = 0.001, 
                              mode = 'min', verbose = 1)


history = model.fit(
    train_generator,
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs = EPOCHS,
    validation_data = validation_generator,
    validation_steps = VALIDATION_STEPS,
    callbacks = [model_save, early_stop, reduce_lr]
)

# Visualizing Results

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
sns.set_style("white")
plt.suptitle('Train history', size = 15)

ax1.plot(epochs, acc, label = "Training acc")
ax1.plot(epochs, val_acc,  label = "Validation acc")
ax1.set_title("Training and validation acc")
ax1.legend()

ax2.plot(epochs, loss,  label = "Training loss")
ax2.plot(epochs, val_loss, label = "Validation loss")
ax2.set_title("Training and validation loss")
ax2.legend()

plt.show()